<a href="https://colab.research.google.com/github/kmk028/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 3

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [0]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt


In [0]:
# read in the data frame
features = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',header = None,names = features)

In [26]:
# figure out non 'a' , 'b' values in dataframe
df[~df['A1'].isin(['a','b'])].head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
248,?,24.50,12.750,u,g,c,bb,4.75,t,t,2,f,g,00073,444,+
327,?,40.83,3.500,u,g,i,bb,0.50,f,f,0,f,s,01160,0,-
346,?,32.25,1.500,u,g,c,v,0.25,f,f,0,t,g,00372,122,-
374,?,28.17,0.585,u,g,aa,v,0.04,f,f,0,f,g,00260,1004,-
453,?,29.75,0.665,u,g,w,v,0.25,f,f,0,t,g,00300,0,-


In [0]:
# replace '?' values with NaN
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data',header = None,names = features, na_values = '?')

In [28]:
df.isnull().sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [29]:
df.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

In [0]:
# Convert all continous features to float
df['A11'] = df['A11'].astype('float')
df['A15'] = df['A15'].astype('float')

In [31]:
df.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11    float64
A12     object
A13     object
A14    float64
A15    float64
A16     object
dtype: object

In [32]:
df.shape

(690, 16)

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [33]:
# data frame of approved applications
approval = df.loc[df['A16'] == '+']
approval.sample(8)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
244,b,18.50,2.000,u,g,i,v,1.50,t,t,2.0,f,g,120.0,300.0,+
208,b,35.17,2.500,u,g,k,v,4.50,t,t,7.0,f,g,150.0,1270.0,+
169,b,37.50,1.125,y,p,d,v,1.50,f,f,0.0,t,g,431.0,0.0,+
576,b,30.17,0.500,u,g,c,v,1.75,t,t,11.0,f,g,32.0,540.0,+
573,a,65.17,14.000,u,g,ff,ff,0.00,t,t,11.0,t,g,0.0,1400.0,+
17,a,23.25,5.875,u,g,q,v,3.17,t,t,10.0,f,g,120.0,245.0,+
122,a,24.75,12.500,u,g,aa,v,1.50,t,t,12.0,t,g,120.0,567.0,+
29,b,42.08,1.040,u,g,w,v,5.00,t,t,6.0,t,g,500.0,10000.0,+


In [34]:
#data frame of rejected applications
rejection = df.loc[df['A16'] == '-']
rejection.sample(8)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
483,b,23.75,0.415,y,p,c,v,0.040,f,t,2.0,f,g,128.0,6.0,-
335,b,27.67,0.750,u,g,q,h,0.165,f,f,0.0,t,g,220.0,251.0,-
290,b,23.00,0.750,u,g,m,v,0.500,f,f,0.0,t,s,320.0,0.0,-
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0.0,f,g,280.0,364.0,-
423,b,26.17,0.835,u,g,cc,v,1.165,f,f,0.0,f,g,100.0,0.0,-
100,b,37.50,1.750,y,p,c,bb,0.250,t,f,0.0,t,g,164.0,400.0,-
544,b,30.08,1.040,y,p,i,bb,0.500,t,t,10.0,t,g,132.0,28.0,-
330,b,20.42,0.000,NaN,NaN,NaN,NaN,0.000,f,f,0.0,f,p,NaN,0.0,-


In [35]:
print(approval.shape)
print(rejection.shape)

(307, 16)
(383, 16)


In [0]:
# Function to calculate t-test p-Value omitting NaNs in process
def t_test(feature):
  stat, pvalue = st.ttest_ind(approval[feature], rejection[feature], nan_policy='omit')
  print (f"Statistic of {feature} is {stat}.")
  print (f"P-Value of {feature} is {pvalue}.")
  return pvalue
  

In [43]:
# Interpretation of p values : to reject or not reject null hypothesis 
#ie, means of approval and rejection dataframe features are same or different and different by how much 
cont_feature = ['A2','A3','A8','A11','A14','A15']
for i in cont_feature:
  if t_test(i) <0.05:
    print('We can reject null hypothesis that 2 features have identical means\n')
  else:
    print('We cannot reject null hypothesis that 2 features have identical means\n')
    
        

Statistic of A2 is 4.2922156166315535.
P-Value of A2 is 2.0276637071781407e-05.
We can reject null hypothesis that 2 features have identical means

Statistic of A3 is 5.52998337614816.
P-Value of A3 is 4.551680702308068e-08.
We can reject null hypothesis that 2 features have identical means

Statistic of A8 is 8.935819983773698.
P-Value of A8 is 3.6710537401601785e-18.
We can reject null hypothesis that 2 features have identical means

Statistic of A11 is 11.667004222431277.
P-Value of A11 is 7.957718568079967e-29.
We can reject null hypothesis that 2 features have identical means

Statistic of A14 is -2.6358251986645476.
P-Value of A14 is 0.008586135473979569.
We can reject null hypothesis that 2 features have identical means

Statistic of A15 is 4.680216020964486.
P-Value of A15 is 3.4520256956287944e-06.
We can reject null hypothesis that 2 features have identical means



In [50]:
# Function to calculate crosstab and find if credidt approval is independant of 
# a particular feature. 
cat_feature = ['A1','A4','A5','A6','A7','A9','A10','A12','A13']

for i in cat_feature:
  crosstab = pd.crosstab(df['A16'], df[i])
  chi_squared, pvalue, _, _ = st.chi2_contingency(crosstab)
  print (f"P-Value of {i} is {pvalue}.")
  print (f"Statistic of {i} is {chi_squared}.")
  if pvalue <0.05:
    print(f"We can say credit approval and {i} are not independant\n")
  else:
    print(f"We cannot say credit approval and {i} are not independant\n")
    

P-Value of A1 is 0.5768937883001117.
Statistic of A1 is 0.31128326491619945.
We cannot say credit approval and A1 are not independant

P-Value of A4 is 2.010680204180363e-06.
Statistic of A4 is 26.234074966202144.
We can say credit approval and A4 are not independant

P-Value of A5 is 2.010680204180363e-06.
Statistic of A5 is 26.234074966202144.
We can say credit approval and A5 are not independant

P-Value of A6 is 3.4999300402715717e-15.
Statistic of A6 is 98.32520342679135.
We can say credit approval and A6 are not independant

P-Value of A7 is 3.625452872372267e-07.
Statistic of A7 is 45.034207140240554.
We can say credit approval and A7 are not independant

P-Value of A9 is 3.1185900878457007e-79.
Statistic of A9 is 355.2038167412799.
We can say credit approval and A9 are not independant

P-Value of A10 is 5.675727374527571e-33.
Statistic of A10 is 143.06956205083145.
We can say credit approval and A10 are not independant

P-Value of A12 is 0.45094587758631943.
Statistic of A12 is

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

*Your words here!*

1. A11 has lowest p-Value(7.957718568079967e-29), which shows means of A11 in approved and rejected cases vary a lot. So this can be one good indicator to predict approval or rejection. 

    A14 has the higest p-Value amongst all features (0.008586135473979569). This still suggests means of A14 in approved and rejected cases are different, but to a lesser extent. So we can say A14 is not a great feature to look at for chances of approval when compared to other features

2. A9 has very low p-value (3.1185900878457007e-79). This suggests that credit approval and A9
are not independant ie, this feature is very differnt for approval cases and rejection cases. 

    A12 has highest p-values among all features. This suggests this feature is pretty similar for both approved and rejected cases. Hence we can say credit approval and A12 are independant

3. Most tough part for me was understanding what is the null hypothesis in each case, ie how to interpret results. 